In [3]:
!pip install -q condacolab
import condacolab
condacolab.install()
!pip install torch torchvision
!pip install d2l==1.0.0b0
!conda install -c conda-forge torchmetrics

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:35
🔁 Restarting kernel...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.4/887.4 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 94.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 107.0 MB/s eta 0:00:00
Looking in indexes: https://p

Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ failed with initial frozen solve. Retrying with flexible solve.
Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / 

In [11]:
!pip install wandb -qU
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: amoseley018. Use `wandb login --relogin` to force relogin


True

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [1]:
import os
from typing import Optional, Sequence
import torch
import torchvision
from torchvision import transforms
from torch import nn, Tensor
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader, Dataset
from torchsummary import summary
from d2l import torch as d2l
from pathlib import Path
from torchmetrics.classification import BinaryF1Score
import sys
import nibabel as nib
import numpy as np
import gc
import skimage
import h5py
import math
import PIL
from PIL import Image

/usr/local/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [2]:
class FocalLoss(nn.Module):
    '''
    Multi-class Focal Loss
    '''
    def __init__(self, gamma=2, weight=None):
        super(FocalLoss, self).__init__()
        self.gamma = gamma

        if weight == None:
            self.weight = [1, 1]
        else:
            self.weight = weight

    def forward(self, input, target):
        """
        input: [N, C], float32
        target: [N, ], int64
        """
        logpt = F.log_softmax(input, dim=0)
        pt = torch.exp(logpt)
        logpt = (1-pt)**self.gamma * logpt
        loss = F.nll_loss(logpt, target, self.weight)
        return loss

In [3]:
class BalancedCELoss(nn.Module):
    def __init__(self, weight=0.5):
        super().__init__()

        self.weight = weight

    def forward(self, input, target):
        loss = 0
        for i, el in enumerate(input):
            loss += (target[i] * torch.log(el) * self.weight) + ((1 - target[i]) * torch.log(1 - el) * (1 - self.weight))

        return -1 * loss / len(input)

In [4]:
class LITSBinaryDataset(Dataset):
    def __init__(self, fileName):
        super().__init__()

        self.file = h5py.File(fileName, 'r')

        self.length = 0

        for scan in list(self.file.keys()):
            self.length += len(list(self.file[scan].keys()))

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        scanNum = 0
        currInd = 0
        scan = list(self.file.keys())[0]
        while currInd + len(list(self.file[scan].keys())) <= idx:
            currInd += len(list(self.file[scan].keys()))
            scanNum += 1
            scan = list(self.file.keys())[scanNum]

        sliceNum = idx - currInd

        data = self.file[list(self.file.keys())[scanNum]]["Slice" + str(sliceNum)]["Slice"]
        label = self.file[list(self.file.keys())[scanNum]]["Slice" + str(sliceNum)].attrs.get("ImageLabel")

        result = []

        result.append(torch.Tensor(data[...]).unsqueeze(0))
        result.append(torch.Tensor(label).squeeze(0))

        return result

    def closeFile(self):
        self.file.close()

In [5]:
class convBlock(nn.Module):
    def __init__(self, inChannels, outChannels, batchNorm, strides, layerMean, layerDev) -> None:
        super().__init__()

        self.conv1 = nn.Conv2d(inChannels, outChannels, kernel_size=3, padding=1, stride=strides)
        self.conv2 = nn.Conv2d(outChannels, outChannels, kernel_size=3, padding=1)

        nn.init.normal_(self.conv1.weight, mean=layerMean, std=layerDev)
        nn.init.normal_(self.conv2.weight, mean=layerMean, std=layerDev)

        if(batchNorm):
            self.bn1 = nn.BatchNorm2d(outChannels)
        else:
            self.bn1 = False

    def forward(self, X):
        Y = self.conv1(X)

        if(self.bn1):
            Y = self.bn1(Y)

        Y = F.relu(Y)

        return torch.Tensor(F.relu(self.conv2(Y)))

In [6]:
class EncoderBlock(nn.Module):
    def __init__(self, inChannels, outChannels, strides) -> None:
        super().__init__()

        self.conv = convBlock(inChannels, outChannels, True, strides, 0, 0.01)
        self.pool = nn.MaxPool2d(2, stride=strides)

    def forward(self, X):

        Y = self.conv.forward(X)

        #return torch.Tensor(self.pool(Y)), Y
        return self.pool(Y)

In [7]:
class DecoderBlock(nn.Module):
    def __init__(self, inChannels, outChannels, strides) -> None:
        super().__init__()

        self.convTrans = nn.ConvTranspose2d(inChannels, outChannels, 2, stride=strides, padding=1)
        self.conv = convBlock(outChannels, outChannels, True, strides, 0, 0.25)

    def forward(self, X, skipFeatures):
        Y = self.convTrans(X)
        Y = torch.cat(X, skipFeatures)
        return self.conv(Y)

In [8]:
def evaluate_accuracy(net, testIter, device=None):
    if isinstance(net, nn.Module):
        net.eval()
        if not device:
            device = next(iter(net.parameters())).device

    metric = d2l.Accumulator(2)

    with torch.no_grad():
        for X, y in testIter:
            X = X.to(device)
            y = y.to(device)

            metric.add(d2l.accuracy(torch.round(torch.clamp(net(X).squeeze(1), min=0, max=1)), y), y.numel())

    return metric[0] / metric[1]

In [9]:
def train(net: nn.Sequential, trainIter, testIter, numEpochs, startEpoch, learnRate, batchSize, device: torch.device, startDim, epochsToDouble, modelFileName, lossFunc = nn.BCEWithLogitsLoss()):
    print(f"Training on {device}")
    
    net.to(device)
    optimizer = torch.optim.SGD(net.parameters(), lr=learnRate)
    numBatches = len(trainIter)

    anim1 = d2l.Animator(xlabel="Epoch", xlim=[0, numEpochs - 1], legend=["Train Accuracy", "Test Accuracy"])
    anim2 = d2l.Animator(xlabel="Epoch", xlim=[0, numEpochs - 1], legend=["Loss"])

    imageFunc = transforms.ToPILImage()
    tensorFunc = transforms.ToTensor()

    currDim = startDim
    for epoch in range(startEpoch, numEpochs):
        if epoch != 0 and epoch % epochsToDouble == 0:
            currDim *= 2
        
        net.train()
        metric = d2l.Accumulator(2)

        for i, (X, y) in enumerate(trainIter):
            optimizer.zero_grad()
            y = y.to(device)

            realX = []

            for slice in X:
                slice = imageFunc(slice.squeeze(0)).resize((currDim, currDim), PIL.Image.BILINEAR)
                slice = tensorFunc(slice).to(device)
                realX.append(slice)

            realX = torch.stack(realX).to(device)

            yhat = torch.sigmoid(net(realX)).squeeze(1)

            prediction = torch.round(torch.clamp(yhat, min=0, max=1)).to(device)

            l = lossFunc(yhat, y.long())
            l.backward()
            optimizer.step()

            metric.add(l, d2l.accuracy(prediction, y) / batchSize)

        torch.save(net.state_dict(), modelFileName + "Epoch" + str(epoch))

        testAcc = evaluate_accuracy(net, testIter, device)

        #print(f"Train Acc: {metric[1] / (numBatches * batchSize)} Test Acc: {testAcc} Loss: {metric[0] / (numBatches * batchSize)}")

        wandb.log({"Train Acc": metric[1] / (numBatches * batchSize),
                   "Test Acc": testAcc,
                   "Loss": (metric[0] / (numBatches * batchSize))
                  })
        anim1.add(epoch, ((metric[1] / (numBatches * batchSize)), testAcc))
        anim2.add(epoch, (metric[0] / (numBatches * batchSize)))
        #print(f"Loss: {sumLoss / numBatches} Accuracy: {sumAcc / (numBatches * batchSize)}")

In [ ]:
gc.collect()

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
#device = torch.device("cpu")

startEpoch = 0
networkFileName = ""
fileSaveName = "/content/drive/MyDrive/LiverClassifierModelRun1"

batchSize = 3
learnRate = 0.00001
epochs = 100
startDim = 32
epochsToDouble = 25

lossFunc = BalancedCELoss(weight=0.7)

wandb.init(project="LiverClassifier",
           name="Run2",
           config={
               "BatchSize":batchSize,
               "LearnRate":learnRate,
               "Epochs":epochs,
               "StartDimension":startDim,
               "EpochsToDouble":epochsToDouble
           })

block1 = EncoderBlock(1, 64, 1)
block2 = EncoderBlock(64, 128, 1)
block3 = EncoderBlock(128, 256, 1)
block4 = EncoderBlock(256, 512, 1)
block5 = EncoderBlock(512, 1024, 1)

net = nn.Sequential(block1, block2, block3, block4, nn.AdaptiveAvgPool2d((1, 1)), nn.Flatten(), nn.Linear(512, 1))
#print(summary(net, (1, 512, 512)))

if networkFileName != "":
    net.load_state_dict(torch.load(networkFileName))

print("Intialized model")

trainDataset = LITSBinaryDataset("drive/MyDrive/TrainDataset.hdf5")
testDataset = LITSBinaryDataset("drive/MyDrive/TestDataset.hdf5")

print("Dataset loaded")

trainIter = DataLoader(trainDataset, batch_size=batchSize, shuffle=True)
testIter = DataLoader(testDataset, batch_size=batchSize)

train(net, trainIter, testIter, epochs, startEpoch, learnRate, batchSize, device, startDim, epochsToDouble, fileSaveName, lossFunc=lossFunc)

trainDataset.closeFile()
testDataset.closeFile()

wandb.finish()

Intialized model
Dataset loaded
Training on cuda
